## Cohort Retention with Python

Retention has been a popular term in the recent development of analytical world. Cohort analysis is a well-known technique to easily grasp the fundamental of how retention works and benefits as a metric. This notebook specifically shows a step-by-step how to create a cohort retention heat map which often used as visual to help analyst better understand the retention rate of particular subject in the business and simplify it to make business team easily understand.

**Data Context**
<br> The data is a daily customer transaction data from a market place company. The data only consists user_id (unique id for each user), order_id (unique id for each purchase transaction made), created_date (user registration date), and trx_date (user transaction date).

**Goal**
<br> To create a monthly user retention heat map. Business team need to know monthly retention rate of users since their registeration.

**Definition**
<br> Based on the context and goal above, we need to define several things, such as:
1. Cohort: is a group of users who share common characteristics around a time period. In this, the cohort is: users who register their account in every month of 2022
2. Retention: is a measure of how well your platform retains users. In this case if 100 people sign ups for the market place on first month, but only 20 of them have transaction in the next month, your "retention rate" is 20%.

**!! Please pay attention to every comment and description in the code box !!**

#### 1. Import required modules

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

#### 2. Import data 
I normally use CSV format. The column arrangement is important to note. You might have various format of data with different column names and arrangement. But I strongly suggest to completely follow my data column naming and arrangement (if you wish to use this notebook entirely). Since a different column naming will result an error when running.

The column should consists of:
1. **Retention subject column**: this column represents an entity in the data in which you wish to calculate its retention. In this case I want to know user retention rate, so the column contains user_id information. (Other possible data for this column: merchant id, streamer id, user phone number etc.)
2. **Retention object column**: this column is an entity in the data that represent the activity of which the retention subject (*user* in this case) stated as retain. In this data the column is orderid. (Other possible data for this column: app session id, login session id, streaming id etc.)
3. **Cohort head column**: this column consists of a period of time (mostly in a date format) that your retention subject will be grouped into. As stated before, my cohort is: users who register their account in every month of 2022. So my cohort head column is created_date (user account registration date). (Other possible data for this column: user first time of transaction, user app installation date etc.)
4. **Retain time column**: this column is simply represents the time when activity in retention object column happen. In this case, the activity is user transaction so the column is trx_date. (Other possible data for this column: user purchase time, post uploaded time etc.).

If you have other columns in your data (usually product_id, purchase_amount, customer_address etc.), you must not include those columns for this cohort retention analysis.

In [20]:
# Import data
user_trans = pd.read_csv('user_transaction_data.csv')
user_trans.head()

,user_id,orderid,created_date,trx_date
0,7fe1001270779f9f0,10f9E4OV,2022-04-03,2022-05-04
1,80b3f013002398420,3f8BBKZU,2022-07-02,2022-12-11
2,7e78801253630ff30,88f9F15X,2022-01-21,2022-06-01
3,80b3f012841758430,3f89FB9N,2022-05-16,2022-06-11
4,7fe11012762342f50,1129GDIX,2022-04-18,2022-07-13


In [3]:
# understand the data type
user_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   user_id       1000000 non-null  object
 1   orderid       1000000 non-null  object
 2   created_date  1000000 non-null  object
 3   trx_date      1000000 non-null  object
dtypes: object(4)
memory usage: 30.5+ MB


#### 3. Convert time data to datetime format
As we can see from the data info above, **created_date** and **trx_date** column treated have *object* datatype. We need to convert those columns into *datetime* datatype.

In [4]:
user_trans['created_date'] = pd.to_datetime(user_trans.created_date)
user_trans['trx_date'] = pd.to_datetime(user_trans.trx_date)
user_trans['created_month'] = pd.to_datetime(user_trans['created_date'].dt.strftime('%Y-%m-01'))
user_trans['trx_month'] = pd.to_datetime(user_trans['trx_date'].dt.strftime('%Y-%m-01'))

In [5]:
user_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   user_id        1000000 non-null  object        
 1   orderid        1000000 non-null  object        
 2   created_date   1000000 non-null  datetime64[ns]
 3   trx_date       1000000 non-null  datetime64[ns]
 4   created_month  1000000 non-null  datetime64[ns]
 5   trx_month      1000000 non-null  datetime64[ns]
dtypes: datetime64[ns](4), object(2)
memory usage: 45.8+ MB


#### 4. Calculate the cohort size
Cohort size is a number of users that belong into each cohort. In this case we need to calculate the number of users who registered their account in each month of 2022.

In [7]:
cohort_size = user_trans.groupby('created_month').user_id.nunique().reset_index()
cohort_size.rename(columns = {'user_id':'cohort_size'}, inplace = True)
cohort_size

,created_month,cohort_size
0,2022-01-01,23713
1,2022-02-01,14452
2,2022-03-01,26827
3,2022-04-01,25264
4,2022-05-01,32144
5,2022-06-01,26467
6,2022-07-01,29770
7,2022-08-01,20740
8,2022-09-01,22397
9,2022-10-01,15679


#### 5. Truncate the date columns
Since we want to get the retention rate in a monthly basis, we need to truncate the date column so it's rounded to the beginning of month (e.g. created_date = "2022-04-24" >> "2022-04-01"). After that you need to apply pandas drop_duplicates as the data must be has lots of duplicates after the truncation. **Note**: you only need 1 user transaction of each month to state that the user is a retained user.

In [9]:
user_trans_trunc = user_trans[['user_id','created_month','trx_month']].drop_duplicates().reset_index().drop(['index'], axis=1)

In [21]:
user_trans_trunc.shape

(573196, 4)

#### 6. Calculate the time difference between trx_month and created_month
At this step, we add a new column called *month_diff* which consists of a month difference between the trx_month and created_month. If the *created_month* is *2022-04-01* and *trx_month* is *2022-07-01*, the month_diff will be 3 (3 months between July and April). It means the user has a transaction in the third month after their registration, and considered as a retained customer in '2022-07-01'.

In [11]:
user_trans_trunc['month_diff'] = user_trans_trunc['trx_month'].dt.to_period('M').astype(int) - user_trans_trunc['created_month'].dt.to_period('M').astype(int)

In [12]:
user_trans_trunc.tail()

,user_id,created_month,trx_month,month_diff
573191,803da0127635318b0,2022-04-01,2022-07-01,3
573192,7fe11012759170c10,2022-04-01,2022-07-01,3
573193,80b3f013015438560,2022-07-01,2022-07-01,0
573194,7fe1001275310f9f0,2022-04-01,2022-07-01,3
573195,80b3f012825178550,2022-05-01,2022-07-01,2


#### 7. Pivot
Let's pivot the data with *month_diff* as columns and *created_month* as rows. The value of each cell will be the count unique of retained users.

In [13]:
monthly_pivot_count = pd.DataFrame(pd.pivot_table(user_trans_trunc, values='user_id', index='created_month', columns='month_diff', aggfunc=lambda x: len(x.unique())).to_records())

In [14]:
monthly_pivot_count

,created_month,0,1,2,3,4,5,6,7,8,9,10,11
0,2022-01-01,14500.0,4103.0,5456.0,4459.0,5040.0,4810.0,5607.0,3986.0,4823.0,4761.0,4478.0,4418.0
1,2022-02-01,3879.0,4931.0,3936.0,4304.0,4104.0,4757.0,3384.0,4010.0,4017.0,3906.0,3786.0,NaN
2,2022-03-01,12426.0,7110.0,7542.0,6976.0,8191.0,4809.0,5392.0,5332.0,5322.0,5090.0,NaN,NaN
3,2022-04-01,9892.0,9432.0,8444.0,9356.0,6288.0,5790.0,5671.0,6075.0,5371.0,NaN,NaN,NaN
4,2022-05-01,15141.0,10610.0,11723.0,7974.0,7996.0,6677.0,7501.0,6272.0,NaN,NaN,NaN,NaN
5,2022-06-01,9522.0,10697.0,6216.0,7493.0,7379.0,6841.0,6639.0,NaN,NaN,NaN,NaN,NaN
6,2022-07-01,12425.0,8987.0,10582.0,10228.0,10171.0,6846.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2022-08-01,5677.0,10120.0,9416.0,10656.0,6556.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2022-09-01,9330.0,10096.0,11110.0,8630.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022-10-01,8819.0,11055.0,9010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 8. Join with the cohort size dataframe
To get the retention rate (in %) we need to compare the number of retained users in each month with the cohort size. At this step, we join the pivot dataframe with cohort size dataframe. As can be seen from the result, there are 23,713 users who registered their account in January 2022, and only 14,500 of them who carried out at least one transaction in the same month.

In [15]:
monthly_pivot_join = pd.merge(
                        monthly_pivot_count,
                        cohort_size,
                        how="inner",
                        on="created_month")

monthly_pivot_join = monthly_pivot_join[['created_month','cohort_size','0','1','2','3','4','5','6','7','8','9','10','11']]
monthly_pivot_join

,created_month,cohort_size,0,1,2,3,4,5,6,7,8,9,10,11
0,2022-01-01,23713,14500.0,4103.0,5456.0,4459.0,5040.0,4810.0,5607.0,3986.0,4823.0,4761.0,4478.0,4418.0
1,2022-02-01,14452,3879.0,4931.0,3936.0,4304.0,4104.0,4757.0,3384.0,4010.0,4017.0,3906.0,3786.0,NaN
2,2022-03-01,26827,12426.0,7110.0,7542.0,6976.0,8191.0,4809.0,5392.0,5332.0,5322.0,5090.0,NaN,NaN
3,2022-04-01,25264,9892.0,9432.0,8444.0,9356.0,6288.0,5790.0,5671.0,6075.0,5371.0,NaN,NaN,NaN
4,2022-05-01,32144,15141.0,10610.0,11723.0,7974.0,7996.0,6677.0,7501.0,6272.0,NaN,NaN,NaN,NaN
5,2022-06-01,26467,9522.0,10697.0,6216.0,7493.0,7379.0,6841.0,6639.0,NaN,NaN,NaN,NaN,NaN
6,2022-07-01,29770,12425.0,8987.0,10582.0,10228.0,10171.0,6846.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2022-08-01,20740,5677.0,10120.0,9416.0,10656.0,6556.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2022-09-01,22397,9330.0,10096.0,11110.0,8630.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022-10-01,15679,8819.0,11055.0,9010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 9. Get the retention rate
This step simply transform the number of monthly retained users into percentage format, dividing them by cohort size. As can be seen from the result, only 61% of the users who registered their account in January 2022 have transaction (retained) in the same month.

In [16]:
monthly_pivot_pct = monthly_pivot_join[['created_month','cohort_size']]
for i in range (0,12):
    monthly_pivot_pct[str(i)] = np.round((monthly_pivot_join[str(i)]/monthly_pivot_join['cohort_size'])*100,0).fillna(0).astype(int)

In [17]:
# Transform to percentage (number of retained users in each month/cohort size)
monthly_pivot_pct = monthly_pivot_pct.fillna(0)
monthly_pivot_pct['created_month'] = monthly_pivot_pct['created_month'].astype(str)
monthly_pivot_pct['cohort_size'] = monthly_pivot_pct['cohort_size'].astype(str)
monthly_pivot_pct

,created_month,cohort_size,0,1,2,3,4,5,6,7,8,9,10,11
0,2022-01-01,23713,61,17,23,19,21,20,24,17,20,20,19,19
1,2022-02-01,14452,27,34,27,30,28,33,23,28,28,27,26,0
2,2022-03-01,26827,46,27,28,26,31,18,20,20,20,19,0,0
3,2022-04-01,25264,39,37,33,37,25,23,22,24,21,0,0,0
4,2022-05-01,32144,47,33,36,25,25,21,23,20,0,0,0,0
5,2022-06-01,26467,36,40,23,28,28,26,25,0,0,0,0,0
6,2022-07-01,29770,42,30,36,34,34,23,0,0,0,0,0,0
7,2022-08-01,20740,27,49,45,51,32,0,0,0,0,0,0,0
8,2022-09-01,22397,42,45,50,39,0,0,0,0,0,0,0,0
9,2022-10-01,15679,56,71,57,0,0,0,0,0,0,0,0,0


#### 10. Coloring
Let's color the retention rate and don't forget to set the min-max value of the rate.

In [18]:
monthly_pivot_pct.style.background_gradient(cmap='Blues',vmin=0,vmax=100)

,created_month,cohort_size,0,1,2,3,4,5,6,7,8,9,10,11
0,2022-01-01,23713,61,17,23,19,21,20,24,17,20,20,19,19
1,2022-02-01,14452,27,34,27,30,28,33,23,28,28,27,26,0
2,2022-03-01,26827,46,27,28,26,31,18,20,20,20,19,0,0
3,2022-04-01,25264,39,37,33,37,25,23,22,24,21,0,0,0
4,2022-05-01,32144,47,33,36,25,25,21,23,20,0,0,0,0
5,2022-06-01,26467,36,40,23,28,28,26,25,0,0,0,0,0
6,2022-07-01,29770,42,30,36,34,34,23,0,0,0,0,0,0
7,2022-08-01,20740,27,49,45,51,32,0,0,0,0,0,0,0
8,2022-09-01,22397,42,45,50,39,0,0,0,0,0,0,0,0
9,2022-10-01,15679,56,71,57,0,0,0,0,0,0,0,0,0


**Highlight**
<br> From the result, we can see that the user retention rate goes lower as the month passed. Even users tend to not purchase anything right after their registration, there are months with the lowest retention rate in the same month of users' registration (February and August, only 27%). This isn't a good sign for the business. The business team should find a way how to attract users to do more frequent purchases and to acquire more potential users. Or they can reflect on what they are doing in October to December 2022, since the retention in that period is improving.